In [44]:
import pandas as pd
import numpy as np

In [45]:
# Adjust the path as per your file structure
player_stats_all_seasons = '/Users/nacho/Desktop/DV7/Pyhton_Coded_Algorithem/data/Processed/player_stats_all_seasons.xlsx'

# Read the Excel files
player_stats_all_seasons = pd.read_excel(player_stats_all_seasons, engine='openpyxl')

In [46]:
def normalizar_estadisticas_por_grupo(df, columnas):
    """
    Normaliza las estadísticas de los jugadores aplicando filtros de temporada y 
    normalizando los valores al rango 0-100.
    """
    # Calcular el umbral de minutos para la temporada 2023
    umbral_2023 = df[df['Temporada'] == 2023]['Minutos'].max() * 0.1

    # Preparar un DataFrame para las columnas normalizadas
    df_norm = pd.DataFrame(index=df.index)

    # Iterar sobre las columnas para normalizar
    for columna in columnas:
        df_norm[columna + '_norm'] = np.nan  # Inicializar con NaN

        # Agrupar por 'Posición_General', 'Temporada', y 'Competición'
        for (posicion, temporada, competicion), group_df in df.groupby(['Posición_General', 'Temporada', 'Competición']):
            # Aplicar condiciones de filtro de la primera función
            if temporada in [2018, 2019, 2020, 2021, 2022]:
                indices = group_df[group_df['Minutos'] >= 400].index
            elif temporada == 2023:
                indices = group_df[group_df['Minutos'] >= umbral_2023].index
            else:
                continue  # Para otras temporadas, no se aplica normalización

            # Calcular z-scores y escalar al rango 0-100 para los índices seleccionados
            if not indices.empty:
                mean = df.loc[indices, columna].mean()
                std = df.loc[indices, columna].std()
                if std > 0:
                    z_scores = (df.loc[indices, columna] - mean) / std
                    min_z = z_scores.min()
                    max_z = z_scores.max()
                    scaled = (z_scores - min_z) / (max_z - min_z) * 100
                    df_norm.loc[indices, columna + '_norm'] = scaled
                else:
                    df_norm.loc[indices, columna + '_norm'] = 50  # Valor neutro si no hay variación

    # Combinar el DataFrame original con las nuevas columnas normalizadas
    df_final = pd.concat([df, df_norm], axis=1)

    return df_final

In [47]:
columnas_a_normalizar = [
    "Minutos", "Faltas x90","Tarjetas Amarillas", "Tarjetas Amarillas x90",
    "Tarjetas Rojas", "Tarjetas Rojas x90", "Faltas Sufridas x90", "Aceleraciones x90", "Carreras Progresivas x90",
    "Goles Concedidos", "Goles Concedidos x90", "Xg Parado", "Xg Parado x90", "Porterías A Cero", "Tiros En Contra", "Tiros En Contra x90", 
    "Paradas x90", "Paradas, %", "Goles Prevenidos", "Salidas Del Portero x90", "Duelos Aéreos Del Portero x90", "Goles Prevenidos x90", 
    "Acciones Defensivas Exitosas x90", "Intercepciones x90", "Intercepciones AjPos x90", "Entradas x90", "Entradas AjPos x90", "Duelos x90", 
    "Duelos Ganados", "Duelos Defensivos Ganados", "Duelos Defensivos x90", "Duelos Aéreos x90", "Duelos Aéreos Ganados", 
    "Duelos Ofensivos Ganados", "Duelos Ofensivos x90", "Pases x90", "Pases Completados, %", "Pases Cortos Y Medios x90", 
    "Pases Cortos Y Medios Completados, %", "Longitud Promedio De Pase", "Pases Hacia Adelante x90", "Pases Hacia Adelante Completados, %",
    "Pases Verticales x90", "Pases Verticales Completados, %", "Pases Clave x90", "Pases Inteligentes x90", "Pases Inteligentes Completados, %", 
    "Pases a Último Tercio x90", "Pases Completados a Último Tercio, %", "Pases al espacio Exitosos, %", "Pases al espacio x90",
    "Pase Profundo Completado x90", "Centros Completados, %", "Pase Progresivo x90", "Pase Progresivo Completados, %", 
    "Pase A Área De Penalti x90", "Pase Preciso A Área De Penalti, %", "Centros x90", "Centros Desde La Derecha x90", 
    "Centros Completados Desde La Derecha, %", "Centros Desdes La Izquierda x90", "Centros Completados Desde La Izquierda, %",
    "Centros al Área x90", "Centros Profundos Completados x90", "Pases Largos x90", "Longitud Promedio De Pase Largo", 
    "Pases Largos Completados, %", "Pases Recibidos x90", "Pases Largos Recibido x90", "Asistencias", "Asistencias x90", "xA", "xA x90",
    "Preasistencia x90", "Pre Preasistencia x90", "Asistencias De Tiro x90", "Pases Hacia Atrás x90", "Pases Hacia Atrás Completados, %", 
    "Pase Hacia Atrás Al Portero x90", "Acciones Ofensivas Completados x90", "Regates x90", "Regates Completados, %", "Tiros", "Tiros x90", 
    "Tiros A Puerta, %", "Xg", "Xg x90", "Goles", "Goles x90", "Goles De Cabeza", "Goles De Cabeza x90", "Toques En El Área x90",
    "Goles sin Penalti", "Goles sin Penalti x90", "Penaltis Tirados", "Penaltis Marcados, %", "Tasa de Conversión de Gol, %", 
    "Tiros Libres Tirados x90", "Tiros Libres Directos Tirados x90", "Tiros Libres Directos A Puerta, %", "Córners Tirados x90"
]

# Llamar a la función con el DataFrame y las columnas especificadas
df_normalized = normalizar_estadisticas_por_grupo(player_stats_all_seasons, columnas_a_normalizar)

/var/folders/31/w995rjws313c_wf9sz9kmcvm0000gn/T/ipykernel_94096/4108186703.py:14: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_norm[columna + '_norm'] = np.nan  # Inicializar con NaN


In [48]:
# Export the processed data to an Excel file
processed_file_path = '/Users/nacho/Desktop/DV7/Pyhton_Coded_Algorithem/data/Normalized/player_stats_all_seasons_normalized.xlsx'
df_normalized.to_excel(processed_file_path, index=False, engine='openpyxl')